In [3]:
from google.colab import files

upload = files.upload()

Saving turkish_movie_sentiment_dataset.csv to turkish_movie_sentiment_dataset.csv


In [4]:
import numpy as np
import pandas as pd

df = pd.read_csv("turkish_movie_sentiment_dataset.csv")

In [6]:
import re
import nltk
import ktrain
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stop_word_list = stopwords.words("turkish")

In [8]:
def preprocess(text):
  text = text.lower()
  text = re.sub(r"\n", "", text)
  text = re.sub(r"\t", "", text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  words = text.split()
  words = [word for word in words if not word in stop_word_list]
  words = [word.strip() for word in words if len(word.strip()) > 1]
  text = " ".join(words)
  return text

In [9]:
df["clean_comment"] = df["comment"].apply(preprocess)

In [10]:
def int_point(point):
  return point[0]

In [12]:
df["target"] = df["point"].apply(int_point)

In [20]:
df["target"].value_counts()

COK_IYI       27463
MUKEMMEL      15873
IYI           14494
IDARE_EDER    13866
KOTU           6381
COK_KOTU       5150
Name: target, dtype: int64

In [15]:
df["target"] = df["target"].astype("str")

In [18]:
df["target"] = df["target"].str.replace("5", "MUKEMMEL")
df["target"] = df["target"].str.replace("4", "COK_IYI")
df["target"] = df["target"].str.replace("3", "IYI")
df["target"] = df["target"].str.replace("2", "IDARE_EDER")
df["target"] = df["target"].str.replace("1", "KOTU")
df["target"] = df["target"].str.replace("0", "COK_KOTU") 

In [33]:
ndf = df.sample(10000)

In [34]:
X = ndf["clean_comment"]
y = ndf["target"]

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [36]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [37]:
classes = np.array(["MUKEMMEL", "COK_IYI", "IYI", "IDARE_EDER", "KOTU", "COK KOTU"])

In [38]:
from ktrain.text import Transformer

transformer = Transformer("burakaytan/roberta-base-turkish-uncased", maxlen=500, classes=classes)

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


In [39]:
train = transformer.preprocess_train(X_train, y_train)
valid = transformer.preprocess_test(X_test, y_test)

preprocessing train...
language: tr
train sequence lengths:
	mean : 36
	95percentile : 113
	99percentile : 299


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: tr
test sequence lengths:
	mean : 37
	95percentile : 120
	99percentile : 321


In [40]:
model = transformer.get_classifier()

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:1093: UserWarning: Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn(


In [41]:
learner = ktrain.get_learner(model, train_data=train, val_data=valid, batch_size=6)

In [46]:
learner.fit_onecycle(0.0001, 1)



begin training using onecycle policy with max lr of 0.0001...
1334/1334 [==============================] - 984s 736ms/step - loss: 1.6464 - accuracy: 0.3372 - val_loss: 1.6606 - val_accuracy: 0.3195


In [47]:
predictor = ktrain.get_predictor(learner.model, preproc=transformer)

In [48]:
predictor.predict("Film gerçekten harikaydi. Bayildim! Efsaneydi")

'COK_IYI'